<a href="https://colab.research.google.com/github/raniii318/GoogleCloudSelector/blob/main/GCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# 🌩 CLOUD SERVICE SELECTOR (Demo)
# ===============================

!pip install pyspark gradio cohere pandas matplotlib

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import gradio as gr
import cohere

# Step 1: Start Spark
spark = SparkSession.builder.appName("CloudServiceSelector").getOrCreate()

# Step 2: Small built-in dataset (no CSV required)
data = [
    ("AWS EC2", 10, 99.99, 0.08),
    ("Azure VM", 15, 99.95, 0.07),
    ("Google Cloud", 12, 99.90, 0.06),
    ("IBM Cloud", 20, 99.80, 0.05),
]
columns = ["service", "latency", "uptime", "cost"]
df = spark.createDataFrame(data, columns)

# Step 3: Feature Engineering
df = df.withColumn("latency_score", 1 / col("latency")) \
       .withColumn("uptime_score", col("uptime") / 100) \
       .withColumn("cost_score", 1 / col("cost")) \
       .withColumn(
           "performance_score",
           0.5 * (1 / col("latency")) + 0.3 * (col("uptime") / 100) + 0.2 * (1 / col("cost"))
       )

pdf = df.toPandas()
print("Processed Cloud Data:")
print(pdf)

# Step 4: (Optional) Add your Cohere API key for natural language explanation
COHERE_API_KEY = "YOUR_COHERE_API_KEY"   # <-- Replace with your real key if you have one
co = cohere.Client(COHERE_API_KEY)

def recommend_service(latency_weight, uptime_weight, cost_weight):
    pdf["custom_score"] = (
        latency_weight * (1 / pdf["latency"]) +
        uptime_weight * (pdf["uptime"] / 100) +
        cost_weight * (1 / pdf["cost"])
    )
    best = pdf.sort_values(by="custom_score", ascending=False).iloc[0]

    prompt = (
        f"Recommend the best cloud service based on these metrics:\n"
        f"Service: {best['service']}\n"
        f"Latency: {best['latency']} ms, Uptime: {best['uptime']}%, Cost: ${best['cost']}/hr.\n"
        f"Explain why this service is a good choice in 2 lines."
    )

    try:
        response = co.generate(model="command", prompt=prompt, max_tokens=50)
        reason = response.generations[0].text.strip()
    except Exception:
        reason = f"{best['service']} performs best overall based on your selected weights."

    return f"✅ Recommended: {best['service']}\n💡 Reason: {reason}"

# Step 5: Gradio Interface
def ui(latency_w, uptime_w, cost_w):
    return recommend_service(latency_w, uptime_w, cost_w)

iface = gr.Interface(
    fn=ui,
    inputs=[
        gr.Slider(0, 1, value=0.5, label="Latency Importance"),
        gr.Slider(0, 1, value=0.3, label="Uptime Importance"),
        gr.Slider(0, 1, value=0.2, label="Cost Importance"),
    ],
    outputs="text",
    title="☁ Cloud Service Selector",
    description="Adjust the weights to get the best cloud service recommendation."
)

iface.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 43.6 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.3
    Uninstalling httpx-sse-0.4.3:
      Successfully uninstalled httpx-sse-0.4.3
Processed Cloud Data:
        service  latency  uptime  cost  latency_score  uptime_score  \
0       AWS EC2       10   99.99  0.08       0.100000        0.9999   
1      Azure VM       15   99.95  0.07       0.066667        0.9995   
2  Google Cloud       12   99.90  0.06       0.083333        0.9990   
3     IBM Cloud       20   99.80  0.05       0.050000        0.9980   

   cost_score  performance_score  
0   12.500000           2.849970  
1   14.285714           3.190326  
2   16.666667           3.674700  
3   20.000000           4.324400  
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=T